In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

size = 14
legendsize = 12

# QAOA

In [ ]:
df = pd.read_csv('../res_qaoa.csv', sep=',')
df['num_qubits'] = df['num_qubits'].astype(int)
df['num_qubits2'] = df['num_qubits'].astype(int)
df.replace(to_replace=["None"], value=0, inplace=True)
df.set_index('num_qubits', inplace=True)
df

In [ ]:
def label_encoding (row):
    if row['considered_following_qubits'] == 1 :
        return 'Only Direct Neighbor'
    elif row['considered_following_qubits'] == 1000 :
        return "All Neighbors"
df["Encoding Prediction"] = df.apply(lambda row: label_encoding(row), axis=1)
df

In [ ]:
df['time_proposed'] = df['time_proposed'].astype(float)
df['time_baseline_O0'] = df['time_baseline_O0'].astype(float)
df['time_baseline_O1'] = df['time_baseline_O1'].astype(float)
df['time_baseline_O2'] = df['time_baseline_O2'].astype(float)
df['time_baseline_O3'] = df['time_baseline_O3'].astype(float)
df['cx_count_proposed'] = df['cx_count_proposed'].astype(int)
df['cx_count_O0'] = df['cx_count_baseline_O0'].astype(int)
df['cx_count_O1'] = df['cx_count_baseline_O1'].astype(int)
df['cx_count_O2'] = df['cx_count_baseline_O2'].astype(int)
df['cx_count_O3'] = df['cx_count_baseline_O3'].astype(int)

df["time_ratio_O0"] = df["time_proposed"]/df["time_baseline_O0"]
df["time_ratio_O1"] = df["time_proposed"]/df["time_baseline_O1"]
df["time_ratio_O2"] = df["time_proposed"]/df["time_baseline_O2"]
df["time_ratio_O3"] = df["time_proposed"]/df["time_baseline_O3"]
df["cx_ratio_O0"] = df['cx_count_proposed'].divide(df['cx_count_O0'])
df["cx_ratio_O3"] = df['cx_count_proposed'].divide(df['cx_count_O3'])
df

In [ ]:
df[df.index<100].cx_ratio_O3.describe()

In [ ]:
x = df#[df.considered_following_qubits==1]
x = x[x.index%1==0][[
    "Encoding Prediction",
    "sample_probability",
    "time_proposed",
    "time_baseline_O0",
    "time_baseline_O1",
    "time_baseline_O2",
    "time_baseline_O3",
    "time_ratio_O0", 
    "time_ratio_O3", 
    "cx_count_proposed",
    "cx_ratio_O0",
    "cx_ratio_O3",
]
]

x["time_baseline_O0"] = x["time_baseline_O0"].round(4)
x["time_baseline_O3"] = x["time_baseline_O3"].round(4)
x["time_proposed"] = x["time_proposed"].round(4)
x["cx_ratio_O0"] = x["cx_ratio_O0"].round(2)
x["cx_ratio_O3"] = x["cx_ratio_O3"].round(2)
x

In [ ]:
x["Benchmark"] = "Qubits=" + x.index.astype(str) + ", p=" + x.sample_probability.astype(str)

In [ ]:
x = x.sort_values(["Encoding Prediction","sample_probability", "num_qubits"])
x.set_index('Benchmark', inplace=True)
x = x.drop('sample_probability', axis=1)
x

In [ ]:
x.time_ratio_O0.describe()

In [ ]:
x.time_ratio_O3.describe()

In [ ]:
x.cx_ratio_O0.describe()

In [ ]:
x.cx_ratio_O3.describe()

In [ ]:
x = x.rename(columns={"time_ratio_with_opt_O0": "Time Ratio O0", 
                      "time_ratio_with_opt_O3": "Time Ratio O3", 
                      "cx_count_baseline_O0": "CX Count Baseline O0",
                      "cx_count_baseline_O3": "CX Count Baseline O3", 
                      "cx_count_with_opt": "CX Count New Scheme", 
                      "cx_ratio_with_opt_O0": "CX Count Ratio O0", 
                      "cx_ratio_with_opt_O3": "CX Count Ratio O3", 
                      "time_new_scheme_with_opt":"Time New Scheme", 
                      "time_baseline_O0":"Time Baseline O0",
                      "time_baseline_O3":"Time Baseline O3"
                     }
            )
print(x.to_latex())

# Graph Trial

In [ ]:
for considered_following_qubits in ["Only Direct Neighbor", "All Neighbors"]:
    for sample_probability in [0.3,0.7]:
        df_new = df[(df.sample_probability==sample_probability) & (df["Encoding Prediction"]==considered_following_qubits)]
        
        ax1 = df_new.plot(x='num_qubits2', y='cx_count_proposed', color='orange', label="Proposed Scheme")   
        ax1.tick_params(which='both', labelsize=size)
        ax2 = df_new.plot( x='num_qubits2', y='cx_count_O0', color='red', ax=ax1, label="Qiskit's O0")
        ax3 = df_new.plot( x='num_qubits2', y='cx_count_O1', color='purple', ax=ax1, label="Qiskit's O1") 
        ax4 = df_new.plot( x='num_qubits2', y='cx_count_O2', color='blue', ax=ax1, label="Qiskit's O2") 
        ax5 = df_new.plot( x='num_qubits2', y='cx_count_O3', color='green', ax=ax1, label="Qiskit's O3") 
        
        plt.xlabel("Qubits", size=size)
        plt.ylabel("CX Count", size=size)
        #plt.xscale("log")
        plt.yscale("log")
        plt.legend(fontsize=legendsize)
        plt.savefig('cx_'+str(considered_following_qubits) + '_'+ str(sample_probability)+'.pdf')
        plt.show()
        
        ax6 = df_new.plot( x='num_qubits2', y='time_proposed', color='orange', label="Proposed Scheme")   
        ax6.tick_params(which='both', labelsize=size)
        ax7 = df_new.plot(x='num_qubits2', y='time_baseline_O0', color='red', ax=ax6, label="Qiskit's O0")
        ax8 = df_new.plot(x='num_qubits2', y='time_baseline_O1', color='purple', ax=ax6, label="Qiskit's O1") 
        ax9 = df_new.plot( x='num_qubits2', y='time_baseline_O2', color='blue', ax=ax6, label="Qiskit's O2") 
        ax10 = df_new.plot( x='num_qubits2', y='time_baseline_O3', color='green', ax=ax6, label="Qiskit's O3") 
        
        plt.xlabel("Qubits", size=size)
        plt.ylabel("Time", size=size)
        #plt.xscale("log")
        plt.yscale("log")
        plt.legend(fontsize=legendsize)
        plt.savefig('time_'+str(considered_following_qubits) + '_'+ str(sample_probability)+'.pdf')
        plt.show()

# Average Value Calculation

In [ ]:
import math
def orderOfMagnitude(number):
    return math.floor(math.log(number, 10))

df["order_magnitudes_diff"] = df["time_proposed"].apply(orderOfMagnitude)-df["time_baseline_O3"].apply(orderOfMagnitude)
df["order_magnitudes_diff"].describe()

In [ ]:
df["cx_ratio_O3"].describe()

# Satellite

In [ ]:
df = pd.read_csv('../res_satellite.csv', sep=',')
df['num_qubits'] = df['num_qubits'].astype(int)
df['num_qubits2'] = df['num_qubits'].astype(int)
df.replace(to_replace=["None"], value=0, inplace=True)
df.set_index('num_qubits', inplace=True)
df

In [ ]:
df['time_proposed'] = df['time_proposed'].astype(float)
df['time_baseline_O0'] = df['time_baseline_O0'].astype(float)
df['time_baseline_O1'] = df['time_baseline_O1'].astype(float)
df['time_baseline_O2'] = df['time_baseline_O2'].astype(float)
df['time_baseline_O3'] = df['time_baseline_O3'].astype(float)
df['cx_count_proposed'] = df['cx_count_proposed'].astype(int)
df['cx_count_O0'] = df['cx_count_baseline_O0'].astype(int)
df['cx_count_O1'] = df['cx_count_baseline_O1'].astype(int)
df['cx_count_O2'] = df['cx_count_baseline_O2'].astype(int)
df['cx_count_O3'] = df['cx_count_baseline_O3'].astype(int)
df

In [ ]:
df["Benchmark"] = "Qubits=" + df.index.astype(str) + ", p=" + df.sample_probability.astype(str)
df = df.sort_values(["sample_probability", "num_qubits"])
df.set_index('Benchmark', inplace=True)
df

In [ ]:
df["cx_ratio_o3"] = df["cx_count_proposed"]/df["cx_count_O3"]
df["cx_ratio_o3"].describe()

In [ ]:
for considered_following_qubits in [1]:

    for sample_probability in [0.4]:
        print(considered_following_qubits, sample_probability)
        df_new = df[(df.sample_probability==sample_probability) & (df.considered_following_qubits==considered_following_qubits)]

        ax1 = df_new.plot(x='num_qubits2', y='cx_count_proposed', color='orange', label="Proposed Scheme")   
        ax1.tick_params(which='both', labelsize=size)
        ax3 = df_new.plot( x='num_qubits2', y='cx_count_O0', color='red', ax=ax1, label="Qiskit's O0")
        ax4 = df_new.plot( x='num_qubits2', y='cx_count_O1', color='purple', ax=ax1, label="Qiskit's O1") 
        ax5 = df_new.plot( x='num_qubits2', y='cx_count_O2', color='blue', ax=ax1, label="Qiskit's O2") 
        ax6 = df_new.plot( x='num_qubits2', y='cx_count_O3', color='green', ax=ax1, label="Qiskit's O3") 


        plt.xlabel("Qubits", size=size)
        plt.ylabel("CX Count", size=size)
        #plt.xscale("log")
        plt.yscale("log")
        plt.legend(fontsize=legendsize)
        plt.savefig('sat_cx_'+ str(sample_probability)+'.pdf')
        plt.show()

        ax5 = df_new.plot( x='num_qubits2', y='time_proposed', color='orange', label="Proposed Scheme")   
        ax5.tick_params(which='both', labelsize=size)
        ax7 = df_new.plot(x='num_qubits2', y='time_baseline_O0', color='red', ax=ax5, label="Qiskit's O0")
        ax8 = df_new.plot(x='num_qubits2', y='time_baseline_O1', color='purple', ax=ax5, label="Qiskit's O1") 
        ax9 = df_new.plot( x='num_qubits2', y='time_baseline_O2', color='blue', ax=ax5, label="Qiskit's O2") 
        ax10 = df_new.plot( x='num_qubits2', y='time_baseline_O3', color='green', ax=ax5, label="Qiskit's O3") 

        plt.xlabel("Qubits", size=size)
        plt.ylabel("Time", size=size)
        #plt.xscale("log")
        plt.yscale("log")
        plt.legend(fontsize=legendsize, loc="center right")
        plt.savefig('sat_time_' + str(sample_probability)+'.pdf')
        plt.show()


In [ ]:
df["order_magnitudes_diff"] = df["time_proposed"].apply(orderOfMagnitude)-df["time_baseline_O3"].apply(orderOfMagnitude)
df["order_magnitudes_diff"].describe()